# Background Interpretation:
The task revolves around analyzing sales data to determine if specific months had lower sales revenues, which could be influenced by weather conditions (temperature). The objective is to:

1. Identify months where the total sales revenue fell below the threshold of $40,000.
2. Calculate the average temperature for these underperforming months, since the company WaterPro believes that weather (temperature) may be a factor affecting sales performance.

 # Steps in the Analysis:
1. Data Cleaning: The raw data provided daily information on WaterCure sales (units sold, unit price, and temperature). The initial rows included irrelevant text, so the dataset was cleaned by skipping unnecessary rows and renaming the columns appropriately.

2. Revenue Calculation: The total daily revenue was computed by multiplying the number of units sold by the unit price. This allowed us to later sum the daily revenues to get monthly totals.

3.  Monthly Aggregation: Since the data was at the daily level, it was grouped by months to calculate:

* The total sales revenue for each month.
* The average temperature for each month.
  
4. Filtering Low-Revenue Months: After calculating monthly revenues, we identified the months where the revenue fell below $40,000.
5. Temperature Analysis: For these underperforming months, we calculated the average temperature using NumPy to help WaterPro understand whether low sales correlate with specific temperature patterns.

In [5]:
import pandas as pd
import numpy as np

# Loading  CSV file
file_path = 'C:\\Users\\balbi\\Downloads\\daily_sales.csv'  # Replace with your actual file path
daily_sales_data_cleaned = pd.read_csv(file_path, skiprows=2)

# Renaming columns for better clarity
daily_sales_data_cleaned.columns = ['Date', 'Daily Units Sold', 'Daily Unit Price', 'Daily Temperature (C)']

# Converting 'Date' column to datetime for easy aggregation by the month
daily_sales_data_cleaned['Date'] = pd.to_datetime(daily_sales_data_cleaned['Date'], format='%m/%d/%Y')

# Ensuring numeric columns are correct  in type
daily_sales_data_cleaned['Daily Units Sold'] = pd.to_numeric(daily_sales_data_cleaned['Daily Units Sold'], errors='coerce')
daily_sales_data_cleaned['Daily Unit Price'] = pd.to_numeric(daily_sales_data_cleaned['Daily Unit Price'], errors='coerce')
daily_sales_data_cleaned['Daily Temperature (C)'] = pd.to_numeric(daily_sales_data_cleaned['Daily Temperature (C)'], errors='coerce')

# Calculating daily sales revenue.
daily_sales_data_cleaned['Daily Revenue'] = daily_sales_data_cleaned['Daily Units Sold'] * daily_sales_data_cleaned['Daily Unit Price']

# Grouping data by month to calculate monthly revenue and average temperature
monthly_data = daily_sales_data_cleaned.groupby(daily_sales_data_cleaned['Date'].dt.to_period('M')).agg({
    'Daily Revenue': 'sum',
    'Daily Temperature (C)': 'mean'
}).reset_index()

# Filtering the months with revenue below 40,000
below_target_months = monthly_data[monthly_data['Daily Revenue'] < 40000].copy()

# Converting Date column back to string for better presentation
below_target_months.loc[:, 'Date'] = below_target_months['Date'].astype(str)

# Printing result
print(below_target_months)


      Date  Daily Revenue  Daily Temperature (C)
5  2024-06        35793.0              15.433333
6  2024-07        35137.0              13.838710
7  2024-08        22200.0              16.225806


# Interpretation:
1. Data Cleaning: The CSV file is loaded, and unnecessary rows are skipped. The columns are renamed for better clarity.
2. Revenue Calculation: Daily revenue is calculated by multiplying the number of units sold by the unit price.
3.Monthly Aggregation: Data is grouped by year and month to calculate total monthly revenue and the average monthly temperature.
4. Filtering: Months where the revenue is below $40,000 are identified.
5. Display: The filtered results are displayed, including the total revenue and average temperature for those months.

The output from the analysis revealed three specific months where the sales revenue fell below the target of $40,000. Here is a summary of those months along with their respective total revenues and average temperatures:

* June 2024: Total Revenue = $35,793, Average Temperature = 15.43°C.

* July 2024: Total Revenue = $35,137, Average Temperature = 13.84°C.

* August 2024: Total Revenue = $22,200, Average Temperature = 16.23°C.

# Summary of Insights:
* Revenue Deficit: The sales revenue for June, July, and August 2024 did not meet the $40,000 target, with August being particularly low.
* Temperature Correlation: These months had relatively mild temperatures, with averages ranging between 13.8°C and 16.2°C. This suggests that cooler months might be linked to lower sales.
* WaterPro can use this information to investigate potential correlations between cooler temperatures and decreased sales, possibly adjusting their strategy to compensate for seasonal fluctuations.

Therefore, the months where the avg temperature is really low, < 16 C, we see the sales dropped below $40k.

In [8]:
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Loading review data
file_path = 'C:\\Users\\balbi\\Downloads\\water_product_reviews_500_actual_names (1).csv' 
reviews_data = pd.read_csv(file_path)

# Initializing SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Functioning to classify sentiment based on compound score
def classify_sentiment(review):
    score = analyzer.polarity_scores(review)['compound']
    if score >= 0.05:
        return 'positive'
    elif score <= -0.05:
        return 'negative'
    else:
        return 'neutral'

#Using sentiment classification to each review
reviews_data['sentiment'] = reviews_data['Review Text'].apply(classify_sentiment)

# Calculating the average rating for each sentiment category
average_rating_by_sentiment = reviews_data.groupby('sentiment')['Rating'].mean()

# Summarizing  total number of positive, neutral, and negative reviews for each month
reviews_data['Review Date'] = pd.to_datetime(reviews_data['Review Date'])
reviews_data['Month'] = reviews_data['Review Date'].dt.to_period('M')

reviews_summary_by_month = reviews_data.groupby(['Month', 'sentiment']).size().unstack(fill_value=0)

# Calculating  average rating for each month
average_rating_by_month = reviews_data.groupby('Month')['Rating'].mean()

output='C:\\Users\\balbi\\Downloads\\Singh_Balbir_Sentiment.csv'
reviews_data.to_csv(output, index=False)

# Print results
print("Average Rating by Sentiment:")
print(average_rating_by_sentiment)

print("\nTotal Number of Positive, Neutral, and Negative Reviews per Month:")
print(reviews_summary_by_month)

print("\nAverage Rating by Month:")
print(average_rating_by_month)


Average Rating by Sentiment:
sentiment
negative    1.451923
neutral     3.000000
positive    4.187500
Name: Rating, dtype: float64

Total Number of Positive, Neutral, and Negative Reviews per Month:
sentiment  negative  neutral  positive
Month                                 
2023-01          13        3        15
2023-02          14        4        10
2023-03          10        3        18
2023-04           6        2        22
2023-05          19        0        12
2023-06          11        1        18
2023-07          17        3        11
2023-08          13        2        16
2023-09          19        0        11
2023-10           9        3        19
2023-11          11        1        18
2023-12          16        2        13
2024-01          16        1        14
2024-02          11        2        16
2024-03          10        4        17
2024-04          10        0        20
2024-05           3        5         6

Average Rating by Month:
Month
2023-01    2.870968
2023-02 

# Interpreation:
* Sentiment Analysis: The SentimentIntensityAnalyzer analyzes each review’s text and categorizes it as positive, neutral, or negative based on the compound score.
* Average Rating Calculation: We calculate the average rating for each sentiment category (positive, neutral, and negative).
* Monthly Sentiment Summary: We summarize the number of reviews for each sentiment category (positive, neutral, negative) per month.
* Monthly Average Rating: The code calculates the average rating for each month.

This provides the overall rating performance for the product per month. For example, if a month has a high average rating (like 4.30), it indicates that customers were generally satisfied during that period. Conversely, a lower rating (like 3.10) could suggest dissatisfaction during that month.

# Summary of the Results:
* Positive Sentiment: Reviews with a positive sentiment tend to have higher ratings (around 4-5 stars), which suggests that customers who leave positive feedback are highly satisfied with the product.

* Neutral Sentiment: Neutral reviews generally have moderate ratings (around 3 stars), indicating that while customers were not entirely unhappy, they were not fully satisfied either.

* Negative Sentiment: Negative reviews typically have lower ratings (around 1-2 stars), indicating dissatisfaction with the product.

* Monthly Trends: Months with more positive reviews are likely to show higher average ratings, while months with more negative reviews may show lower average ratings.


Based on these results, WaterPro can analyze customer satisfaction trends over time and take steps to improve product quality or address specific issues that customers highlight in their negative reviews. If certain months show a spike in negative reviews, WaterPro might investigate possible causes such as seasonal issues, shipping delays, or product defects during that period. Similarly, consistently positive feedback in certain months could indicate successful marketing campaigns, improved customer service, or product enhancements.

WaterPro should aim to reduce the number of negative reviews and increase customer satisfaction by improving areas that customers commonly highlight in their reviews. The company can also monitor how product sentiment changes over time and adjust its strategies accordingly to maintain a positive customer experience.